In [10]:
import numpy as np
import pandas as pd
import math
import random
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge

from sklearn import ensemble  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.neural_network import MLPRegressor 

In [11]:
import pandas as pd

iowa_file_path = './merged_gw.csv'

df = pd.read_csv(iowa_file_path)

df.columns
Position = {"GK" : "0", "MID" : "2", "DEF" : "1", "FWD" : "3"}
  
df["position_num"] = df["position"].map(Position)
df.head()


,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position_num
0,Nathan Redmond,MID,Southampton,1.5,0,0,3,0,0.0,403,...,0.0,1,0,0,0,55,False,0,1,2
1,Junior Stanislas,MID,Bournemouth,1.1,0,0,3,0,0.0,58,...,0.0,1,0,0,0,50,True,0,1,2
2,Armando Broja,FWD,Chelsea,2.0,0,0,3,0,0.3,150,...,19.0,1,0,0,0,55,False,0,1,3
3,Fabian Schär,DEF,Newcastle,2.4,0,3,43,1,14.6,366,...,25.0,15,0,0,0,45,True,0,1,1
4,Jonny Evans,DEF,Leicester,1.9,0,0,15,0,1.3,249,...,0.0,1,0,0,0,45,True,0,1,1


In [12]:
id_df =pd.read_csv('./player_idlist.csv')
id_df['name'] = id_df['first_name'] + ' ' + id_df['second_name']
id_df.drop(['first_name', 'second_name'], axis=1, inplace=True)


	
id_dict = id_df.set_index('name').to_dict()['id']
df.replace({"name": id_dict},inplace=True)
df.head()


,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position_num
0,403,MID,Southampton,1.5,0,0,3,0,0.0,403,...,0.0,1,0,0,0,55,False,0,1,2
1,58,MID,Bournemouth,1.1,0,0,3,0,0.0,58,...,0.0,1,0,0,0,50,True,0,1,2
2,150,FWD,Chelsea,2.0,0,0,3,0,0.3,150,...,19.0,1,0,0,0,55,False,0,1,3
3,366,DEF,Newcastle,2.4,0,3,43,1,14.6,366,...,25.0,15,0,0,0,45,True,0,1,1
4,249,DEF,Leicester,1.9,0,0,15,0,1.3,249,...,0.0,1,0,0,0,45,True,0,1,1


# **Decision Tree Model**
Without split <br>
`MAE = 0.000109`

In [13]:
y=df.xP
desc=[
    'name',
    'minutes',
    'value',
    'threat',
    'position_num',
    'bps',
    'bonus',
    'opponent_team',
    'saves',
    'creativity',
    'ict_index',
    'influence',
    'was_home',
    'assists',
    'goals_conceded',
    'goals_scored',
    'minutes'
]
x=df[desc]

from sklearn.tree import DecisionTreeRegressor
iowa_model = DecisionTreeRegressor(random_state=1)
iowa_model.fit(x, y)
predictions = iowa_model.predict(x)
print(predictions[:20])
print(mean_absolute_error(y, predictions))

[1.5 1.1 2.  2.4 1.9 1.3 1.1 0.5 1.5 1.  1.5 0.5 1.9 0.3 5.5 1.5 0.5 0.
 2.5 1.4]
0.00010919414719371071


In [14]:
x

,name,minutes,value,threat,position_num,bps,bonus,opponent_team,saves,creativity,ict_index,influence,was_home,assists,goals_conceded,goals_scored,minutes
0,403,1,55,0.0,2,3,0,18,0,0.0,0.0,0.0,False,0,0,0,1
1,58,1,50,0.0,2,3,0,2,0,0.0,0.0,0.0,True,0,0,0,1
2,150,15,55,19.0,3,3,0,8,0,0.3,2.5,5.2,False,0,0,0,15
3,366,90,45,25.0,1,43,3,16,0,14.6,10.6,66.0,True,0,0,1,90
4,249,90,45,0.0,1,15,0,4,0,1.3,1.5,14.0,True,0,2,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,441,0,43,0.0,2,0,0,11,0,0.0,0.0,0.0,True,0,0,0,0
9154,436,0,45,0.0,1,0,0,11,0,0.0,0.0,0.0,True,0,0,0,0
9155,538,21,44,0.0,1,1,0,5,0,0.0,0.0,0.0,False,0,0,0,21
9156,119,0,45,0.0,2,0,0,2,0,0.0,0.0,0.0,True,0,0,0,0


# **Decision Tree Model** 
with split <br> 
`MAE = 0.9942`

In [15]:

train_X, val_X, train_y, val_y = train_test_split(x, y, random_state = 0,test_size=0.25)
model = DecisionTreeRegressor()
model.fit(train_X, train_y)

val_predictions = model.predict(val_X)
print(val_predictions[:20])
print(mean_absolute_error(val_y, val_predictions))

[ 0.   3.8  0.2  0.  -0.5  0.   3.4  3.2  0.3  0.   0.   0.   1.5  1.3
  2.5  0.   1.7  0.   1.2  1. ]
0.9789737991266375


# **Random Forest Model**
without split <br> 
`MAE = 0.2741`

In [16]:

regr = RandomForestRegressor(oob_score = True, n_estimators = 10000, max_features = 5)

regr.fit(x, y)
predict = regr.predict(x)
predict

array([1.25734, 1.2715 , 1.69599, ..., 0.40354, 0.22315, 2.72056])

In [17]:
print(mean_absolute_error(y, predict))

0.27426413104207625


# **Random Forest Model**
with spilt <br>
`MAE = 0.77954`


In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
regr = RandomForestRegressor(oob_score = True, n_estimators = 10000, max_features = 5)

regr.fit(X_train, y_train)
predict_train = regr.predict(X_train)

predict_test = regr.predict(X_test)

out_of_bag_predict = regr.oob_score

out_of_bag_predict




True

In [19]:
print(mean_absolute_error(y_test, predict_test))
print(mean_absolute_error(y_train, predict_train))

0.7388343640179856
0.27633482983499225


In [20]:
X_test

,name,minutes,value,threat,position_num,bps,bonus,opponent_team,saves,creativity,ict_index,influence,was_home,assists,goals_conceded,goals_scored,minutes
6785,2,90,45,0.0,0,15,0,11,3,0.0,2.3,23.4,False,0,2,0,90
43,494,0,40,0.0,1,0,0,11,0,0.0,0.0,0.0,False,0,0,0,0
128,311,12,70,2.0,2,3,0,19,0,2.7,0.9,4.4,False,0,0,0,12
9119,619,45,50,0.0,3,4,0,18,0,1.5,0.2,0.4,False,0,1,0,45
7096,516,90,45,6.0,1,19,0,10,0,1.3,2.3,15.2,True,0,4,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564,122,0,40,0.0,1,0,0,9,0,0.0,0.0,0.0,False,0,0,0,0
2917,123,11,53,4.0,3,-1,0,9,0,0.3,0.4,0.0,False,0,0,0,11
453,386,0,40,0.0,1,0,0,15,0,0.0,0.0,0.0,False,0,0,0,0
118,397,27,60,0.0,3,6,0,15,0,0.4,0.4,3.2,False,0,1,0,27


In [21]:
y_test

6785    3.8
43      1.5
128     3.4
9119    1.5
7096    0.2
       ... 
2564    0.0
2917    0.9
453     0.0
118     1.3
6316    0.0
Name: xP, Length: 916, dtype: float64

## Predicting Algorithm Team


In [23]:
df = pd.read_csv('./players_raw.csv')
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,clean_sheets_per_90,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,0,5,100,100,0,0.00000,58822,None,NaN,...,193,2,5778,121,25733,220,0.2,0.5,Cédric,0
1,3,7,253,None,None,6,0.46312,84450,None,NaN,...,38,62,691852,17429,465676,34932,0.5,12.2,Xhaka,2
2,0,0,21,100,100,0,0.00000,153256,None,NaN,...,187,4,149828,8263,206840,6517,0.2,1.0,Elneny,0
3,0,0,14,None,None,0,0.00000,156074,None,NaN,...,197,4,5719,123,15979,247,0.1,1.0,Holding,0
4,0,4,191,100,100,7,0.65693,167199,None,NaN,...,93,42,78659,2614,96954,3185,1.0,8.8,Partey,1


In [24]:
df.drop(df[df['chance_of_playing_next_round'] =='None' ].index, inplace = True)
df.drop(df[df['chance_of_playing_next_round'] =='0' ].index, inplace = True)


In [25]:
df.columns


Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'clean_sheets_per_90',
       'code', 'corners_and_indirect_freekicks_order',
       'corners_and_indirect_freekicks_text', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'creativity', 'creativity_rank', 'creativity_rank_type',
       'direct_freekicks_order', 'direct_freekicks_text', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points',
       'expected_assists', 'expected_assists_per_90',
       'expected_goal_involvements', 'expected_goal_involvements_per_90',
       'expected_goals', 'expected_goals_conceded',
       'expected_goals_conceded_per_90', 'expected_goals_per_90', 'first_name',
       'form', 'form_rank', 'form_rank_type', 'goals_conceded',
       'goals_conceded_per_90', 'goals_scored', 'ict_index', 'ict_index_rank',
       'ict_index_rank_type', 'id', 'in_dreamteam

In [26]:
df['points_per_mil']=df['total_points']/(df['now_cost']/10)
df.head()

ppm_df = df[['points_per_mil','id']]
ppm_df = ppm_df.sort_values(by=['points_per_mil'], ascending=False)
ppm_df.head()

,points_per_mil,id
478,16.271186,357
316,13.829787,258
487,13.673469,366
293,13.260870,346
341,12.222222,225


In [34]:
id_dict_rev = id_df.set_index('id').to_dict()['name']
#id_dict_rev[357]
#id_dict_rev[258]
id_dict_rev[366]
#id_dict_rev[346]

'Fabian Schär'